### NLP Model

The **medium** NLP model of spacy will be used:<br>
o The small one cannot do meaningful comparisons<br>
o The large one is too big for simple textual comparisons (300MB is 10x larger than medium).

In [2]:
import en_core_web_md

nlp_md = en_core_web_md.load()

### Text Features

Import `items`

In [3]:
import pandas as pd

items = pd.read_csv('../datasets/slimmed/items.csv')
items.head()

,title,features,description,videos,details,images,parent_asin,categories,average_rating,rating_number,main_category,store,price
0,Phantasmagoria: A Puzzle of Flesh,['Windows 95'],[],[],"{'Best Sellers Rank': {'Video Games': 137612, ...",[{'thumb': 'https://m.media-amazon.com/images/...,B00069EVOG,"['Video Games', 'PC', 'Games']",4.1,18,Video Games,Sierra,NaN
1,NBA 2K17 - Early Tip Off Edition - PlayStation 4,['The #1 rated NBA video game simulation serie...,['Following the record-breaking launch of NBA ...,[{'title': 'NBA 2K17 - Kobe: Haters vs Players...,"{'Release date': 'September 16, 2016', 'Best S...",[{'thumb': 'https://m.media-amazon.com/images/...,B00Z9TLVK0,"['Video Games', 'PlayStation 4', 'Games']",4.3,223,Video Games,2K,58.0
2,Nintendo Selects: The Legend of Zelda Ocarina ...,['Authentic Nintendo Selects: The Legend of Ze...,[],[],"{'Best Sellers Rank': {'Video Games': 51019, '...",[{'thumb': 'https://m.media-amazon.com/images/...,B07SZJZV88,"['Video Games', 'Legacy Systems', 'Nintendo Sy...",4.9,22,Video Games,Amazon Renewed,37.42
3,"Spongebob Squarepants, Vol. 1",['Bubblestand: SpongeBob shows Patrick and Squ...,['Now you can watch the wild underwater antics...,[],"{'Release date': 'August 15, 2004', 'Best Sell...",[{'thumb': 'https://m.media-amazon.com/images/...,B0001ZNU56,"['Video Games', 'Legacy Systems', 'Nintendo Sy...",4.4,32,Video Games,Majesco,33.98
4,eXtremeRate Soft Touch Top Shell Front Housing...,['Compatibility Models: Ultra fits for Xbox On...,[],[],"{'Best Sellers Rank': {'Video Games': 48130, '...",[{'thumb': 'https://m.media-amazon.com/images/...,B07H93H878,"['Video Games', 'Xbox One', 'Accessories', 'Fa...",4.5,3061,Video Games,eXtremeRate,17.59


In [11]:
# text_items = items[['title', 'features', 'description', 'categories']]
text_items = items[['title', 'features', 'description', 'categories', 'details']]

text_items.head()

,title,features,description,categories,details
0,Phantasmagoria: A Puzzle of Flesh,['Windows 95'],[],"['Video Games', 'PC', 'Games']","{'Best Sellers Rank': {'Video Games': 137612, ..."
1,NBA 2K17 - Early Tip Off Edition - PlayStation 4,['The #1 rated NBA video game simulation serie...,['Following the record-breaking launch of NBA ...,"['Video Games', 'PlayStation 4', 'Games']","{'Release date': 'September 16, 2016', 'Best S..."
2,Nintendo Selects: The Legend of Zelda Ocarina ...,['Authentic Nintendo Selects: The Legend of Ze...,[],"['Video Games', 'Legacy Systems', 'Nintendo Sy...","{'Best Sellers Rank': {'Video Games': 51019, '..."
3,"Spongebob Squarepants, Vol. 1",['Bubblestand: SpongeBob shows Patrick and Squ...,['Now you can watch the wild underwater antics...,"['Video Games', 'Legacy Systems', 'Nintendo Sy...","{'Release date': 'August 15, 2004', 'Best Sell..."
4,eXtremeRate Soft Touch Top Shell Front Housing...,['Compatibility Models: Ultra fits for Xbox On...,[],"['Video Games', 'Xbox One', 'Accessories', 'Fa...","{'Best Sellers Rank': {'Video Games': 48130, '..."


We will want to compare titles, features, and descriptions of items. We start by concatenating all of them in a single series.

In [56]:
N = 20_000
subset_items = text_items.sample(n=N, random_state=42)

#### Accumulating Text Features

In [57]:
titles = subset_items['title']
titles

53664    5-Piece Switch Accessories Bundle compatible f...
19818    Karaoke Revolution American Idol Encore 2 (Ren...
14496     Call of Duty Advanced Warfare - Day Zero Edition
49794    WARFRAME: SARYN PRIME ACCESSORIES PACK - PS4 [...
80739    Inside Europe - Big Hidden Object Game [Download]
                               ...                        
24714          FiGPiN Classic: Marvel Classics - Hulk #499
79484    Power a Skylanders Swap Force Mini pro Ex Wire...
79930                                       Space Invaders
65725    Wii Remote Controller with Motion Plus Compati...
76881                                Dragon Warrior I & II
Name: title, Length: 20000, dtype: object

In [58]:
def concat_array_strings(str_array: str):
	return ', '.join(str_array.replace('[', '').replace(']', '').split(','))

In [59]:
features = subset_items['features'].apply(concat_array_strings)
features

53664    'BUNDLE AND SAVE; $30+ value bundle if bought ...
19818    '40 all new hit songs from the 70s to today', ...
14496    'B00MU1YENG There is a newer version of this i...
49794                                                     
80739    '25 Fun Packed Hidden Picture Levels',  '10 Hi...
                               ...                        
24714                                                     
79484    'Combat design keeps you comfortable during lo...
79930    'Classic alien shooting action on-the-go',  'S...
65725                                                     
76881                                                     
Name: features, Length: 20000, dtype: object

In [60]:
descriptions = subset_items['description'].apply(concat_array_strings)
descriptions

53664                                                     
19818                                                     
14496                                                     
49794    "ACCESS EXCLUSIVE PRIME GEAR AND DISCOUNTED PL...
80739    'PlayHOG presents',  'Inside Europe',  ',  a H...
                               ...                        
24714    'Overview',  'FiGPiN Marvel Classic - The Incr...
79484    "Explore Skylands in style! The officially lic...
79930                                                     
65725    'Specifications:Color: WhiteRemote size: 5.7 x...
76881    'Amazon.com',  'Back in 1989, ',  'Dragon Warr...
Name: description, Length: 20000, dtype: object

In [61]:
details = subset_items['details']
details

53664                                                   {}
19818    {'Best Sellers Rank': {'Video Games': 101098, ...
14496                                                   {}
49794    {'Release date': 'February 16, 2016', 'Pricing...
80739    {'Release date': 'July 11, 2016', 'Pricing': '...
                               ...                        
24714    {'Release date': 'August 6, 2021', 'Pricing': ...
79484    {'Best Sellers Rank': {'Video Games': 160837, ...
79930                                                   {}
65725    {'Best Sellers Rank': {'Video Games': 120498, ...
76881    {'Release date': 'September 21, 2000', 'Best S...
Name: details, Length: 20000, dtype: object

In [62]:
categories = subset_items['categories'].apply(concat_array_strings)
categories

53664    'Video Games',  'Nintendo Switch',  'Accessories'
19818    'Video Games',  'Legacy Systems',  'Nintendo S...
14496    'Video Games',  'Legacy Systems',  'Xbox Syste...
49794    'Video Games',  'PlayStation Digital Content',...
80739                       'Video Games',  'PC',  'Games'
                               ...                        
24714                                                     
79484    'Video Games',  'Legacy Systems',  'Xbox Syste...
79930    'Video Games',  'Legacy Systems',  'Nintendo S...
65725    'Video Games',  'Legacy Systems',  'Nintendo S...
76881    'Video Games',  'Legacy Systems',  'Nintendo S...
Name: categories, Length: 20000, dtype: object

### Creating Text Embeddings

In [63]:
nlp_titles = [(title, nlp_md(title).vector) for title in titles[:N].fillna('').values]

In [64]:
nlp_features = [(feature, nlp_md(feature).vector) for feature in features[:N].fillna('').values]

In [65]:
nlp_descriptions = [(description, nlp_md(description).vector) for description in descriptions[:N].fillna('').values]

In [66]:
nlp_details = [(detail, nlp_md(detail).vector) for detail in details[:N].fillna('').values]

In [67]:
nlp_categories = [(category, nlp_md(category).vector) for category in categories[:N].fillna('').values]

In [68]:
def create_embedding_series(nlp_items):
    _, vectors = zip(*nlp_items)
    return pd.Series(data=vectors)

In [69]:
title_embeddings = create_embedding_series(nlp_titles)

In [70]:
feature_embeddings = create_embedding_series(nlp_features)

In [71]:
description_embeddings = create_embedding_series(nlp_descriptions)

In [80]:
detail_embeddings = create_embedding_series(nlp_details)

In [72]:
category_embeddings = create_embedding_series(nlp_categories)

In [132]:
final_embeddings = (
	0.25 * title_embeddings +
	0.15 * feature_embeddings +
	0.10 * detail_embeddings +
	0.35 * description_embeddings +
	0.15 * category_embeddings
)

In [133]:
final_embeddings = pd.DataFrame(final_embeddings.tolist())
final_embeddings.columns = [f'dim_{i}' for i in range(final_embeddings.shape[1])]

In [134]:
final_embeddings

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,-0.422971,0.182773,-0.024435,-0.029188,0.051011,-0.037062,0.040254,-0.085221,0.010359,0.517289,...,0.089960,0.058149,-0.062360,0.019249,0.014615,0.011827,0.014033,-0.020826,-0.099801,0.063892
1,-0.421952,0.188910,0.050988,-0.053078,0.054547,-0.066487,-0.003701,-0.026202,0.056096,0.618047,...,0.071904,0.067743,-0.020487,0.010016,0.047018,0.040173,-0.016511,-0.078189,-0.089287,0.054977
2,-0.411304,0.156349,-0.003332,-0.042222,-0.002717,-0.038352,0.028354,-0.135241,0.043674,0.760879,...,0.042401,0.067639,-0.105396,-0.010870,0.021127,0.074805,-0.005830,-0.058990,-0.106715,0.014398
3,-0.549883,0.162567,0.001224,-0.061787,0.061685,0.004426,0.019660,-0.148648,0.072335,0.739047,...,0.021569,0.048530,-0.116162,-0.020540,-0.014949,0.016681,-0.064421,-0.100742,-0.035194,0.089746
4,-0.633212,0.155298,-0.053892,-0.013280,0.027510,0.021093,0.037524,-0.174859,0.116670,1.023387,...,0.068321,0.123579,-0.133490,0.005748,0.033466,0.097816,0.037370,-0.053026,-0.069028,0.052830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-0.443515,0.102806,-0.008507,-0.031525,0.024339,0.008350,0.017988,-0.113913,0.018501,0.542736,...,0.000355,0.023091,-0.068736,-0.019705,-0.034403,0.019957,-0.003558,-0.086698,-0.022648,0.012937
19996,-0.675883,0.148179,-0.018593,-0.054282,-0.066229,-0.002043,0.047945,-0.148417,0.052712,0.984539,...,0.005328,0.048390,-0.083039,-0.017146,0.000234,0.092104,-0.007795,-0.119034,-0.075586,0.123557
19997,-0.404340,0.015634,-0.003293,-0.171339,0.030021,-0.082995,-0.097701,0.033466,-0.025181,0.735077,...,-0.071243,0.002920,-0.116886,0.084812,-0.103188,0.145751,-0.008646,0.027607,-0.050045,-0.029290
19998,-0.582627,0.191306,-0.040281,-0.082490,-0.005927,-0.062879,-0.001091,-0.150356,0.076784,0.841456,...,-0.004518,-0.011171,-0.098406,-0.071866,-0.003804,0.046255,-0.013709,-0.086333,-0.027989,0.022030


In [135]:
# import umap
# import matplotlib.pyplot as plt

# # UMAP defaults: n_neighbors=15, min_dist=0.1, n_components=2
# reducer = umap.UMAP(n_components=2, random_state=42)
# umap_results = reducer.fit_transform(final_embeddings)

# # Plot
# plt.figure(figsize=(8, 6))
# plt.scatter(umap_results[:, 0], umap_results[:, 1], cmap='tab10', s=20)
# plt.title("UMAP Projection with Cluster Coloring")
# plt.xlabel("Component 1")
# plt.ylabel("Component 2")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [136]:
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt

# import plotly.express as px
# import pandas as pd

# n_components = 2

# # Apply t-SNE to reduce to desired dimensions
# tsne = TSNE(n_components=n_components, random_state=42, perplexity=40, learning_rate=100, max_iter=500)
# tsne_results = tsne.fit_transform(final_embeddings)

# if n_components == 2:
# 	# Plot in 2D
# 	plt.figure(figsize=(6, 5))
# 	plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c='blue')
		
# 	plt.title("t-SNE Visualization of Embeddings")
# 	plt.xlabel("Component 1")
# 	plt.ylabel("Component 2")
# 	plt.grid(True)
# 	plt.tight_layout()
# 	plt.show()
# else:
# 	# Plot in 3D
# 	df_tsne = pd.DataFrame(tsne_results, columns=['x', 'y', 'z'])
# 	fig = px.scatter_3d(df_tsne, x='x', y='y', z='z', color_discrete_sequence=['blue'])
# 	fig.update_layout(title="3D t-SNE Visualization", scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
# 	fig.show()

In [137]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans

# import plotly.express as px

# # Step 1: Cluster the vectors
# n_clusters = 5  # You can change this
# kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# cluster_labels = kmeans.fit_predict(final_embeddings)

# # Step 2: Plot with cluster colors
# if n_components == 2:
# 	plt.figure(figsize=(10, 9))
# 	scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=cluster_labels, cmap='tab10', s=100)

# 	plt.title("t-SNE with Cluster Coloring")
# 	plt.xlabel("Component 1")
# 	plt.ylabel("Component 2")
# 	plt.grid(True)
# 	plt.tight_layout()
# 	plt.colorbar(scatter, label='Cluster')
# 	plt.show()
# else:
# 	# Step 1: Cluster the vectors
# 	n_clusters = 5
# 	kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# 	cluster_labels = kmeans.fit_predict(final_embeddings)

# 	# Step 3: Prepare DataFrame for plotly
# 	df_tsne = pd.DataFrame(tsne_results, columns=['x', 'y', 'z'])
# 	df_tsne['cluster'] = cluster_labels.astype(str)  # convert to str for discrete color

# 	# Step 4: Plot
# 	fig = px.scatter_3d(
# 		df_tsne, x='x', y='y', z='z',
# 		color='cluster',
# 		color_discrete_sequence=px.colors.qualitative.Set1,
# 		title="3D t-SNE Visualization with Cluster Coloring"
# 	)

# 	fig.update_layout(
# 		scene=dict(
# 			xaxis_title='Component 1',
# 			yaxis_title='Component 2',
# 			zaxis_title='Component 3'
# 		),
# 		legend_title='Cluster'
# 	)

# 	fig.show()

In [138]:
from sklearn.metrics.pairwise import cosine_similarity

sim_mat = cosine_similarity(final_embeddings)
sim_mat

array([[0.9999999 , 0.9351364 , 0.94883144, ..., 0.8336716 , 0.909739  ,
        0.87274796],
       [0.9351364 , 1.0000006 , 0.92092913, ..., 0.82609206, 0.90287995,
        0.8918571 ],
       [0.94883144, 0.92092913, 1.0000002 , ..., 0.8485042 , 0.9109155 ,
        0.8827423 ],
       ...,
       [0.8336716 , 0.82609206, 0.8485042 , ..., 0.9999996 , 0.8393992 ,
        0.8354496 ],
       [0.909739  , 0.90287995, 0.9109155 , ..., 0.8393992 , 1.0000006 ,
        0.90362966],
       [0.87274796, 0.8918571 , 0.8827423 , ..., 0.8354496 , 0.90362966,
        1.0000005 ]], dtype=float32)

In [144]:
test_item_idx = 1
items.loc[test_item_idx]

title              NBA 2K17 - Early Tip Off Edition - PlayStation 4
features          ['The #1 rated NBA video game simulation serie...
description       ['Following the record-breaking launch of NBA ...
videos            [{'title': 'NBA 2K17 - Kobe: Haters vs Players...
details           {'Release date': 'September 16, 2016', 'Best S...
images            [{'thumb': 'https://m.media-amazon.com/images/...
parent_asin                                              B00Z9TLVK0
categories                ['Video Games', 'PlayStation 4', 'Games']
average_rating                                                  4.3
rating_number                                                   223
main_category                                           Video Games
store                                                            2K
price                                                          58.0
Name: 1, dtype: object

In [ ]:
import numpy as np

items.loc[np.argsort(-sim_mat[test_item_idx])[1:10]]

,title,features,description,videos,details,images,parent_asin,categories,average_rating,rating_number,main_category,store,price
117,Mastex Mouse Jiggler with Drive-Free and USB P...,['【Perfect for working from home】Mouse Mover c...,[],"[{'title': 'The Mouse Jiggler for Women', 'url...",{'Product Dimensions': '5.1 x 3.7 x 1.2 inches...,[{'thumb': 'https://m.media-amazon.com/images/...,B0BLXXGZJL,"['Video Games', 'PC', 'Accessories', 'Gaming M...",4.2,91,All Electronics,Mastex,21.99
17913,Daydayup Switch Case Compatible with Nintendo ...,['Fit Nintendo Switch Lite Original Charger AC...,[],"[{'title': 'Excellent!', 'url': 'https://www.a...",{'Package Dimensions': '9.21 x 4.92 x 2.83 inc...,[{'thumb': 'https://m.media-amazon.com/images/...,B08P7R3W6D,"['Video Games', 'Legacy Systems', 'Nintendo Sy...",4.8,6209,All Electronics,daydayup,19.99
10172,targeal 60% Mechanical Gaming Keyboard - 61 Ke...,['60% Compact Layout - Targeal gaming keyboard...,[],"[{'title': 'Great Keyboard!', 'url': 'https://...",{'Product Dimensions': '11.8 x 4.3 x 1.8 inche...,[{'thumb': 'https://m.media-amazon.com/images/...,B0C34DJS6C,"['Video Games', 'PC', 'Accessories', 'Gaming K...",4.1,53,Computers,targeal,44.99
13411,CoBak Carrying Case for Nintendo Switch Lite -...,['Super Case for Nintendo Switch Lite: Durable...,[],"[{'title': 'Nintendo switch lite V3', 'url': '...",{'Product Dimensions': '9.45 x 4.72 x 1.57 inc...,[{'thumb': 'https://m.media-amazon.com/images/...,B09NCT8XV4,"['Video Games', 'Nintendo Switch', 'Accessorie...",4.8,2901,Cell Phones & Accessories,CoBak,15.95
14883,Skylanders SWAP Force Character Dark Wash Buck...,['skylanders swap force'],['Refurbished figure'],[],"{'Best Sellers Rank': {'Video Games': 95239, '...",[{'thumb': 'https://m.media-amazon.com/images/...,B00FV328TW,"['Video Games', 'Legacy Systems', 'PlayStation...",4.0,26,Video Games,ACTIVISION,49.88
7525,eXtremeRate Scary Party Replacement Part Facep...,['Compatibility Model: Ultra fits for Xbox Ser...,[],"[{'title': '💯', 'url': 'https://www.amazon.com...","{'Best Sellers Rank': {'Video Games': 19308, '...",[{'thumb': 'https://m.media-amazon.com/images/...,B09PG31Q5Z,"['Video Games', 'Xbox One']",4.6,3439,Video Games,eXtremeRate,15.99
14866,ORDA PS4 Controller Wireless Game Compatible w...,['ORDA wireless controller is widely compatibl...,[],[],"{'Best Sellers Rank': {'Video Games': 85730, '...",[{'thumb': 'https://m.media-amazon.com/images/...,B09HGZKBMS,"['Video Games', 'PlayStation 4', 'Accessories'...",4.5,19,Video Games,ORDA,NaN
581,5-Star Compatible High Yield Inkjet Cartridge ...,['5-Star Compatible High Yield Inkjet Cartridg...,['5-Star Compatible High Yield Inkjet Cartridg...,[],{'Pricing': 'The strikethrough price is the Li...,[{'thumb': 'https://m.media-amazon.com/images/...,B00T9AHVZE,"['Video Games', 'Legacy Systems', 'Xbox System...",3.8,23,Video Games,5 Star,NaN
7541,Teamwolf Mechanical Gaming Keyboard RGB 104 Fu...,['High-performance mechanical Blue switch: The...,[],[{'title': 'Redragon K556 PRO Upgraded Wireles...,{'Product Dimensions': '17.13 x 4.92 x 1.46 in...,[{'thumb': 'https://m.media-amazon.com/images/...,B08YY573TV,"['Video Games', 'PC', 'Accessories', 'Gaming K...",4.4,127,Computers,Team Wolf,55.99
